# Problem Definition:

Given the dataset, Appliances Energy Consumption, we are to develop a multiple linear regression model that can predict new energy consumption of an appliance when given new test data. To answer some questions, we will need to normalize the dataset using the MinMaxScaler after removing the following columns: [“date”, “lights”]. The target variable is “Appliances”. Use a 70-30 train-test set split with a random state of 42 (for reproducibility).

# Importing Libraries

In [1]:
import pandas as pd
import matplotlib.pyplot  as plt
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


# Importing required libraries for modelling and model evaluation

from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In C:\Users\sky\Anaconda4\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\sky\Anaconda4\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\sky\Anaconda4\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\Users\sky\Anaconda4\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\sky\Anaconda4\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
Th

# Loading the Dataset

In [2]:
# loading the dataset
df = pd.read_csv('energydata_complete.csv')
df.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


Descriptive summary of the dataset

In [3]:
df.describe()

,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
count,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,...,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000
mean,97.694958,3.801875,21.686571,40.259739,20.341219,40.420420,22.267611,39.242500,20.855335,39.026904,...,19.485828,41.552401,7.411665,755.522602,79.750418,4.039752,38.330834,3.760707,24.988033,24.988033
std,102.524891,7.935988,1.606066,3.979299,2.192974,4.069813,2.006111,3.254576,2.042884,4.341321,...,2.014712,4.151497,5.317409,7.399441,14.901088,2.451221,11.794719,4.194648,14.496634,14.496634
min,10.000000,0.000000,16.790000,27.023333,16.100000,20.463333,17.200000,28.766667,15.100000,27.660000,...,14.890000,29.166667,-5.000000,729.300000,24.000000,0.000000,1.000000,-6.600000,0.005322,0.005322
25%,50.000000,0.000000,20.760000,37.333333,18.790000,37.900000,20.790000,36.900000,19.530000,35.530000,...,18.000000,38.500000,3.666667,750.933333,70.333333,2.000000,29.000000,0.900000,12.497889,12.497889
50%,60.000000,0.000000,21.600000,39.656667,20.000000,40.500000,22.100000,38.530000,20.666667,38.400000,...,19.390000,40.900000,6.916667,756.100000,83.666667,3.666667,40.000000,3.433333,24.897653,24.897653
75%,100.000000,0.000000,22.600000,43.066667,21.500000,43.260000,23.290000,41.760000,22.100000,42.156667,...,20.600000,44.338095,10.408333,760.933333,91.666667,5.500000,40.000000,6.566667,37.583769,37.583769
max,1080.000000,70.000000,26.260000,63.360000,29.856667,56.026667,29.236000,50.163333,26.200000,51.090000,...,24.500000,53.326667,26.100000,772.300000,100.000000,14.000000,66.000000,15.500000,49.996530,49.996530


# Data Wrangling

In [4]:
# dropping the lights column since there are more zero entries as seen from the descriptive statistics  above
df.drop(['lights'], axis=1,inplace=True)

# dropping the date column since its not a time series problem
df.drop(['date'], axis=1, inplace=True)

In [5]:
df

,Appliances,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,60,19.890000,47.596667,19.200000,44.790000,19.790000,44.730000,19.000000,45.566667,17.166667,...,17.033333,45.5300,6.600000,733.5,92.000000,7.000000,63.000000,5.300000,13.275433,13.275433
1,60,19.890000,46.693333,19.200000,44.722500,19.790000,44.790000,19.000000,45.992500,17.166667,...,17.066667,45.5600,6.483333,733.6,92.000000,6.666667,59.166667,5.200000,18.606195,18.606195
2,50,19.890000,46.300000,19.200000,44.626667,19.790000,44.933333,18.926667,45.890000,17.166667,...,17.000000,45.5000,6.366667,733.7,92.000000,6.333333,55.333333,5.100000,28.642668,28.642668
3,50,19.890000,46.066667,19.200000,44.590000,19.790000,45.000000,18.890000,45.723333,17.166667,...,17.000000,45.4000,6.250000,733.8,92.000000,6.000000,51.500000,5.000000,45.410389,45.410389
4,60,19.890000,46.333333,19.200000,44.530000,19.790000,45.000000,18.890000,45.530000,17.200000,...,17.000000,45.4000,6.133333,733.9,92.000000,5.666667,47.666667,4.900000,10.084097,10.084097
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19730,100,25.566667,46.560000,25.890000,42.025714,27.200000,41.163333,24.700000,45.590000,23.200000,...,23.200000,46.7900,22.733333,755.2,55.666667,3.333333,23.666667,13.333333,43.096812,43.096812
19731,90,25.500000,46.500000,25.754000,42.080000,27.133333,41.223333,24.700000,45.590000,23.230000,...,23.200000,46.7900,22.600000,755.2,56.000000,3.500000,24.500000,13.300000,49.282940,49.282940
19732,270,25.500000,46.596667,25.628571,42.768571,27.050000,41.690000,24.700000,45.730000,23.230000,...,23.200000,46.7900,22.466667,755.2,56.333333,3.666667,25.333333,13.266667,29.199117,29.199117
19733,420,25.500000,46.990000,25.414000,43.036000,26.890000,41.290000,24.700000,45.790000,23.200000,...,23.200000,46.8175,22.333333,755.2,56.666667,3.833333,26.166667,13.233333,6.322784,6.322784


# Data Preprocessing

In [6]:
# Applying MinMaxScaler to scale the features column 

# instantiate the scaler
scaler = MinMaxScaler()

# normalizing the features
normalized_features = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

In [7]:
# splitting data into features(predictors) and target(response) variables

# predictors
features = normalized_features.drop(['Appliances'], axis=1)

# target variable
target = normalized_features['Appliances']

In [8]:
# splitting data into training and testing set

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.3, random_state = 42)

In [9]:
# create a dictionary of different algorithms 
models = {'Ridge': Ridge(),
        'Lasso': Lasso(),
        'LinearRegression':LinearRegression()
        }

In [10]:
# helper funtion to compute the r2_score, RSS, RMSE, MAE, MSE on the testing set

def compute_score(models, X_train, X_test, y_train, y_test):
    """
    This function iteratively goes through all the models defined in the dictionary and 
    computes the r2_score, MSE, MAE and RMSE.
    
    Parameters: model, training set(X_train), test_set(X_test), train_labels(y_train), and test_labels(y_test).
    
    Returns: This funtion returns a dataFrame containing calculations of each models.
    """
    
    # store properties of each model
    model_properties = []
    
    # loop through the dictionary of models
    for reg_name, regressor in models.items():
        # empty dict for storing properties of each regression models
        reg_dict = {}
        # store the name of each model
        reg_dict['Name'] = reg_name
        # fit the regressor model
        regressor.fit(X_train, y_train)
        # compute the RSS
        reg_dict['RSS'] = round(np.sum(np.square(y_test - regressor.predict(X_test))), 2)
        # compute the r2_score
        reg_dict['r2_score'] = round(r2_score(y_test, regressor.predict(X_test)), 2)
        # compute the MAE
        reg_dict['MAE'] = round(mean_absolute_error(y_test, regressor.predict(X_test)), 2)
        #compute the mean_squared_error
        reg_dict['MSE'] = round(mean_squared_error(y_test, regressor.predict(X_test)), 3)
        # compute the RMSE
        reg_dict['RMSE'] = round(np.sqrt(mean_squared_error(y_test, regressor.predict(X_test))), 3)
        # append the properties of a each regressor to the model_properties list after every iteration
        model_properties.append(reg_dict)
        
        # create a dataframe with a list of all the model properties
    summary_df = pd.DataFrame(model_properties)
    
        
    return summary_df

In [11]:
# execute the function
compute_score(models, X_train, X_test, y_train, y_test)

,Name,RSS,r2_score,MAE,MSE,RMSE
0,Ridge,45.42,0.15,0.05,0.008,0.088
1,Lasso,53.28,-0.00,0.06,0.009,0.095
2,LinearRegression,45.35,0.15,0.05,0.008,0.088


# from question 12 and above:

Question 12: From the dataset, fit a linear model on the relationship between the temperature in the living room in Celsius (x = T2) and the temperature outside the building (y = T6). What is the R^2 value in two D.P?¶

In [12]:
# Obtaining the Training set of the T2 and T6 columns from the normalised data
train_x = X_train[['T2']]
train_y = X_train[['T6']]

# Obtaining the Testing set of the T2 and T6 column from the normalised data  
test_x = X_test[['T2']]
test_y = X_test[['T6']]

# Instantiate the LinearRegression model
model = LinearRegression()

# Fit the model with training set
model.fit(train_x, train_y)

# Make predictions on test set
prediction = model.predict(test_x)


#Compute the r2_score on test set
R2_score = round(r2_score(test_y, prediction), 2)
R2_score

0.64

Question 13: What is the mean absolute error

In [13]:
# Lets instantiate the general LinearRegression Model to answer questions 13 - 17

# Instantiate the LinearRegression Model
lin_reg = LinearRegression()

# fit the model with training set
lin_reg.fit(X_train, y_train)

# Make predictions on testing set
y_pred = lin_reg.predict(X_test)

In [14]:
# Computing the Mean_absolute_error
MAE = round(mean_absolute_error(y_test, y_pred), 2)

MAE

0.05

Question 14: What is the Residual Sum of Squares(in two decimal places)

In [15]:
# Computing the Residual Sum of Squares in 2D.p
RSS = round(np.sum(np.square(y_test - y_pred)), 2)
RSS

45.35

In [16]:
RMSE = round(np.sqrt(mean_squared_error(y_test, y_pred)), 3)

RMSE

0.088

Question 16: What is the Coefficient of Determination(in two decimal places)

In [17]:
# Computing the r2_score
R2_score = r2_score(y_test, y_pred)
round(R2_score, 2)

0.15

Question 17: Obtain the feature weights from your linear model above. Which features have the lowest and highest weights respectively?
Answer: We can see from the Table below that the features with the lowest and highest weights are RH_2, RH_1 respectively.¶

In [18]:
def get_weights_df(model, feat, col_name):
  #this function returns the weight of every feature
  weights = pd.Series(model.coef_, feat.columns).sort_values()
  weights_df = pd.DataFrame(weights).reset_index()
  weights_df.columns = ['Features', col_name]
  weights_df[col_name].round(3)
  return weights_df

# Execute the get_weight function and store in dataframe
linear_weights_df = get_weights_df(lin_reg, X_train, 'Linear_weight')

linear_weights_df

,Features,Linear_weight
0,rv1,-2.759603e+10
1,RH_2,-4.567008e-01
2,T_out,-3.218684e-01
3,T2,-2.361850e-01
4,T9,-1.899320e-01
5,RH_8,-1.575972e-01
6,RH_out,-7.768139e-02
7,RH_7,-4.461288e-02
8,RH_9,-3.979516e-02
9,T5,-1.565763e-02


Question 18: Train a Ridge Regression model with an alpha value of 0.4. Is there any change to the root mean squared error(RMSE) when evaluated on the test set?

Answer: From the calculations below, there are no differences, hence the answer is No

In [19]:
# Instantuate ridge model with default aplha value
ridge = Ridge()

# Fitting the ridge model with training set
ridge.fit(X_train, y_train)

# making predictions on test set
pred = ridge.predict(X_test)

ridge_rmse = round(np.sqrt(mean_squared_error(y_test, pred)), 3)

print(f'The RMSE score for the rigde model with default aplha value when evaluated on the test set is: {ridge_rmse}')


# Instantiate another ridge model with aplha value set to 0.4
ridge_reg = Ridge(alpha=0.4)

# fit on traing set
ridge_reg.fit(X_train, y_train)

# Make new predictions on test set
new_pred = ridge_reg.predict(X_test)

ridge_rmse_new = round(np.sqrt(mean_squared_error(y_test, new_pred)), 3)

print(f'The RMSE score for the rigde model with default aplha value when evaluated on the test set is: {ridge_rmse_new}')

print('There is no change in the RMSE scores')

The RMSE score for the rigde model with default aplha value when evaluated on the test set is: 0.088
The RMSE score for the rigde model with default aplha value when evaluated on the test set is: 0.088
There is no change in the RMSE scores


Question 19: Train a Lasso regression model with an aplha value of 0.001 and obtain the new feature weights with it. How many of the features have non-zero feature weights?

Answer: 4

In [20]:
# Instatiate a lasso regressor with alpha value of 0.001
lasso_reg = Lasso(alpha=0.001)

# Fit the lasso model with training set
lasso_reg.fit(X_train, y_train)

# Make predictions on test set
lasso_preds = lasso_reg.predict(X_test)

# defining a function to get weights
def get_weights_df(model, feat, col_name):
  #this function returns the weight of every feature
  weights = pd.Series(model.coef_, feat.columns).sort_values()
  weights_df = pd.DataFrame(weights).reset_index()
  weights_df.columns = ['Features', col_name]
  weights_df[col_name].round(3)
  return weights_df

lasso_weights_df = get_weights_df(lasso_reg, X_train, 'Lasso_weight')

# Lets obtain the dataFrame with non-zero feature weights
non_zero_weights = lasso_weights_df[lasso_weights_df['Lasso_weight'] != 0]

# view non_zero dataframe
print(non_zero_weights)

print('')

# print the total number of non_zero weights
print(f'The total features with non-zero feature weights are equal to: {len(non_zero_weights)}')

     Features  Lasso_weight
0      RH_out     -0.049557
1        RH_8     -0.000110
24  Windspeed      0.002912
25       RH_1      0.017880

The total features with non-zero feature weights are equal to: 4


Question 20: What is the new RMSE with Lasso Regression (in 3 decimal places)

Answer: 0.094

In [21]:
# Compute the new RMSE with lasso regression
RMSE_new = round(np.sqrt(mean_squared_error(y_test, lasso_preds)), 3)

RMSE_new

0.094